# MongoDB with pymongo -- primer

The following introduction in several basic commands are highly influenced by the guide found on the official documentary webpage https://docs.mongodb.com/getting-started/python/introduction/

After loading the module we connect to a client. Here we use the default local one, localhost:27017. Of course it could be a remote server as well.

In [1]:
from pymongo import MongoClient
client = MongoClient()
print(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


Next we use pymongo to assign a database (groups of collections) called 'tutorial'.

In [2]:
#db = client.test # attribute style
db = client['tutorial'] # dictionary style
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'tutorial')


Next we construct a collection (which corresponds to tables in relational databases, groups of documents) in the database.

In [3]:
#coll = db.dataset # attribute style
coll = db['dataset'] # dictionary style
print(coll)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'tutorial'), 'dataset')


Remark: if the database doesn't exist already, MongoDB creates it automatically.

# Queries

Now we have the collection with specifier restaurants in our database. We can send some queries using pymongo and the method find(). It returns results in a cursor. find() without arguments returns all documents in the collection. We can specify some argument if we are interested in just particular documents/restaurants. Let's say we want to have all steak houses in NYC

In [4]:
cursor = db.restaurants.find({'cuisine': 'Steak'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db72212f48e386d88248'), 'address': {'building': '178', 'coord': [-73.96252129999999, 40.7098035], 'street': 'Broadway', 'zipcode': '11211'}, 'borough': 'Brooklyn', 'cuisine': 'Steak', 'grades': [{'date': datetime.datetime(2014, 3, 8, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 9, 28, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 3, 26, 0, 0), 'grade': 'A', 'score': 3}, {'date': datetime.datetime(2012, 9, 10, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2011, 8, 15, 0, 0), 'grade': 'A', 'score': 13}], 'name': 'Peter Luger Steakhouse', 'restaurant_id': '40364335'}
{'_id': ObjectId('59f9db73212f48e386d88255'), 'address': {'building': '845', 'coord': [-73.965531, 40.765431], 'street': 'Lexington Avenue', 'zipcode': '10065'}, 'borough': 'Manhattan', 'cuisine': 'Steak', 'grades': [{'date': datetime.datetime(2014, 3, 26, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 3, 21, 0, 0), 'grade'

That is probably a little bit two much though. So let's search for steak houses in the Lexington Avenue only (logically AND). Note the dot notation for the embedded property street.

In [5]:
cursor = db.restaurants.find({'cuisine': 'Steak', 'address.street': 'Lexington Avenue'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db73212f48e386d88255'), 'address': {'building': '845', 'coord': [-73.965531, 40.765431], 'street': 'Lexington Avenue', 'zipcode': '10065'}, 'borough': 'Manhattan', 'cuisine': 'Steak', 'grades': [{'date': datetime.datetime(2014, 3, 26, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 3, 21, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2012, 10, 18, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 5, 7, 0, 0), 'grade': 'A', 'score': 3}, {'date': datetime.datetime(2011, 5, 17, 0, 0), 'grade': 'A', 'score': 5}], 'name': "Donohue'S Steak House", 'restaurant_id': '40364572'}
{'_id': ObjectId('59f9db74212f48e386d8c25d'), 'address': {'building': '1032', 'coord': [-73.9617109, 40.771567], 'street': 'Lexington Avenue', 'zipcode': '10021'}, 'borough': 'Manhattan', 'cuisine': 'Steak', 'grades': [{'date': datetime.datetime(2014, 6, 24, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 4, 2, 0, 0), 'grade': 

We can use a comparison operator in queries as well, here we want one pizza place with a score > 30

In [6]:
cursor = db.restaurants.find_one({'grades.score': {'$gt': 30}, 'cuisine': 'Pizza'})
print(cursor)

{'_id': ObjectId('59f9db73212f48e386d8826e'), 'address': {'building': '4035', 'coord': [-73.9395182, 40.8422945], 'street': 'Broadway', 'zipcode': '10032'}, 'borough': 'Manhattan', 'cuisine': 'Pizza', 'grades': [{'date': datetime.datetime(2014, 2, 10, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 2, 4, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2012, 1, 4, 0, 0), 'grade': 'A', 'score': 6}, {'date': datetime.datetime(2011, 9, 15, 0, 0), 'grade': 'C', 'score': 60}], 'name': 'Como Pizza', 'restaurant_id': '40365280'}


Exercise: search for crappy (with a score less than 5) italian or french restaurants in Queens.

In [ ]:
cursor = db.restaurants.find({'$and':[{'$or': [{'cuisine': 'Italian'}, {'cuisine': 'French'}]} \
                                      ,{'borough': 'Queens'}, {'grades.score': {'$lt': 5}}]})
for documents in cursor:
    print(documents)

We can sort the output using the following pymongo routines

In [8]:
import pymongo
cursor = db.restaurants.find({'borough': 'Manhattan', 'cuisine': 'Irish'}).sort([
    ('address.zipcode', pymongo.ASCENDING),
    ('address.coord[0]', pymongo.DESCENDING)
])
for document in cursor:
    print(document)

{'_id': ObjectId('59f9db73212f48e386d8829f'), 'address': {'building': '137', 'coord': [-73.98926, 40.7509054], 'street': 'West   33 Street', 'zipcode': '10001'}, 'borough': 'Manhattan', 'cuisine': 'Irish', 'grades': [{'date': datetime.datetime(2014, 8, 15, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2014, 1, 21, 0, 0), 'grade': 'A', 'score': 5}, {'date': datetime.datetime(2013, 7, 24, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 5, 31, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2012, 1, 26, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2011, 10, 11, 0, 0), 'grade': 'A', 'score': 5}], 'name': 'Blarney Rock', 'restaurant_id': '40366379'}
{'_id': ObjectId('59f9db73212f48e386d882c1'), 'address': {'building': '421', 'coord': [-73.99682299999999, 40.753182], 'street': '9 Avenue', 'zipcode': '10001'}, 'borough': 'Manhattan', 'cuisine': 'Irish', 'grades': [{'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'A', 'scor

Some exercise query and sort

# Manipulations of documents in the collection

Insert a new document with an artificial restaurant.

In [12]:
cursor = db.restaurants.find({'name': 'Space Cookie'})
for documents in cursor:
    print(documents)

from datetime import datetime
new_restaurant = {
        "address": {
            "street": "Broadway",
            "zipcode": "07087",
            "building": "4711",
            "coord": [-74.017212, 40.779185]
        },
        "borough": "Manhattan",
        "cuisine": "Cypermeals",
        "grades": [
            {
                "date": datetime.strptime("2017-11-10", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            }
        ],
        "name": "Space Cookie",
        "restaurant_id": "87654321"
    }
new_one = db.restaurants.insert_one(new_restaurant)
print(new_one)
print(new_one.inserted_id)
cursor = db.restaurants.find({'name': 'Space Cookie'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9e0ab4c174922618035d5'), 'address': {'street': 'Broadway', 'zipcode': '07087', 'building': '4711', 'coord': [-74.017212, 40.779185]}, 'borough': 'Manhattan', 'cuisine': 'Cypermeals', 'grades': [{'date': datetime.datetime(2017, 11, 10, 0, 0), 'grade': 'A', 'score': 11}], 'name': 'Space Cookie', 'restaurant_id': '87654321'}
59f9e0b74c174922618035d6
{'_id': ObjectId('59f9e0ab4c174922618035d5'), 'address': {'street': 'Broadway', 'zipcode': '07087', 'building': '4711', 'coord': [-74.017212, 40.779185]}, 'borough': 'Manhattan', 'cuisine': 'Cypermeals', 'grades': [{'date': datetime.datetime(2017, 11, 10, 0, 0), 'grade': 'A', 'score': 11}], 'name': 'Space Cookie', 'restaurant_id': '87654321'}
{'_id': ObjectId('59f9e0b74c174922618035d6'), 'address': {'street': 'Broadway', 'zipcode': '07087', 'building': '4711', 'coord': [-74.017212, 40.779185]}, 'borough': 'Manhattan', 'cuisine': 'Cypermeals', 'grades': [{'date': datetime.datetime(2017, 11, 10, 0, 0), 'grade': 'A', 'score':

Update a document, here: change Panchito's meal style from Mexican to Spanish

In [13]:
cursor = db.restaurants.find({'name': 'Panchito\'S'})
for documents in cursor:
    print(documents)
result = db.restaurants.update_one({'name': 'Panchito\'S'}, {'$set': {'cuisine': 'Spanish'}})
print('after the update')
cursor = db.restaurants.find({'name': 'Panchito\'S'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db73212f48e386d88272'), 'address': {'building': '103', 'coord': [-74.001043, 40.729795], 'street': 'Macdougal Street', 'zipcode': '10012'}, 'borough': 'Manhattan', 'cuisine': 'Mexican', 'grades': [{'date': datetime.datetime(2014, 5, 22, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 10, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 3, 20, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2012, 5, 17, 0, 0), 'grade': 'B', 'score': 20}], 'name': "Panchito'S", 'restaurant_id': '40365348'}
after the update
{'_id': ObjectId('59f9db73212f48e386d88272'), 'address': {'building': '103', 'coord': [-74.001043, 40.729795], 'street': 'Macdougal Street', 'zipcode': '10012'}, 'borough': 'Manhattan', 'cuisine': 'Spanish', 'grades': [{'date': datetime.datetime(2014, 5, 22, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 10, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 3, 20, 0, 

We can also update multiple documents at once.

In [14]:
result = db.restaurants.update_many(
    {"address.zipcode": "10016", "cuisine": "Other"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": True}
    }
)
print(result.matched_count)
print(result.modified_count)

20
20


Note: MongoDB has a build-in functionality to update if the document exists or to create a new one with the flag {upsert:True}.

Remove a document (here we do the process in single steps, could also use find_one_and_delete to combine the action).

In [15]:
cursor = db.restaurants.find({'name': 'Windjammers Bar'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db73212f48e386d883d3'), 'address': {'building': '552', 'coord': [-73.9070012, 40.7090773], 'street': 'Grandview Avenue', 'zipcode': '11385'}, 'borough': 'Queens', 'cuisine': 'American', 'grades': [{'date': datetime.datetime(2014, 9, 9, 0, 0), 'grade': 'B', 'score': 14}, {'date': datetime.datetime(2014, 1, 7, 0, 0), 'grade': 'B', 'score': 18}, {'date': datetime.datetime(2012, 11, 29, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2011, 12, 14, 0, 0), 'grade': 'A', 'score': 10}], 'name': 'Windjammers Bar', 'restaurant_id': '40377908'}


In [16]:
db.restaurants.delete_one({'name': 'Windjammers Bar'})
cursor = db.restaurants.find_one({'name': 'Windjammers Bar'})
print(cursor)

None


# Data aggregation

If we are interested in the number of restaurants in the single borough we can aggregate them.

In [23]:
cursor = db.restaurants.aggregate([
    {'$group': {'_id': '$borough', 'count': {'$sum': 1}}}])
for document in cursor:
    print(document)

{'_id': 'Staten Island', 'count': 969}
{'_id': 'Brooklyn', 'count': 6086}
{'_id': 'Queens', 'count': 5655}
{'_id': 'Bronx', 'count': 2338}


exersise: group by cuisine

In [18]:
cursor = db.restaurants.aggregate([
    {'$group': {'_id': '$cuisine', 'count': {'$sum': 1}}}])
for document in cursor:
    print(document)

{'_id': 'Chilean', 'count': 1}
{'_id': 'Hawaiian', 'count': 3}
{'_id': 'Southwestern', 'count': 9}
{'_id': 'Hotdogs/Pretzels', 'count': 16}
{'_id': 'Australian', 'count': 16}
{'_id': 'Salads', 'count': 45}
{'_id': 'Czech', 'count': 6}
{'_id': 'Bangladeshi', 'count': 36}
{'_id': 'Category To Be Determined', 'count': 20}
{'_id': 'Pakistani', 'count': 31}
{'_id': 'Peruvian', 'count': 68}
{'_id': 'Soups & Sandwiches', 'count': 51}
{'_id': 'Nuts/Confectionary', 'count': 6}
{'_id': 'CafÃ©/Coffee/Tea', 'count': 2}
{'_id': 'Vietnamese/Cambodian/Malaysia', 'count': 66}
{'_id': 'Afghan', 'count': 14}
{'_id': 'Brazilian', 'count': 26}
{'_id': 'Juice, Smoothies, Fruit Salads', 'count': 273}
{'_id': 'Filipino', 'count': 26}
{'_id': 'Tapas', 'count': 28}
{'_id': 'Chinese/Japanese', 'count': 59}
{'_id': 'Fruits/Vegetables', 'count': 7}
{'_id': 'Creole', 'count': 24}
{'_id': 'Indonesian', 'count': 8}
{'_id': 'Portuguese', 'count': 8}
{'_id': 'Sandwiches', 'count': 459}
{'_id': 'Other', 'count': 991}
{

Delete documents, here all the entries that are listed with 'Missing' for the borough key.

In [22]:
result = db.restaurants.delete_many({"borough": "Missing"})
result.deleted_count

0